### Recurrent Neural Network: 순환신경망

### CBOW: Continuous Back Of Words
- 조건부 확률 <br>

context 먼저 뽑았을 때 그 다음에 label 뽑게 될 확률


벡터로 단어를 얼마나 쉽게 만드ㄴ는지  <br>


활성화함수로 sigmoid아닌 하이퍼볼릭탄젠트 사용. 

- 삼각함수

하이퍼볼릭 사인함스, 하이퍼볼릵 코사인, 하이퍼볼릭 탄젠트... <br>
계산 <br>
도함수 유도하기... <br>


수치미분 안쓰고 역전파 쓰는 이유: 계산비용 대폭 절감위해 <br>
컴퓨터가 빠르게 계산할 수 있는게 덧셈&곱셈 <br>

Q1. 왜 하이퍼볼릭 탄젠트 앞에 싸인 코싸인 탄젠트 이런게 붙냐?<br>
Q2. 왜 하이퍼볼릭(쌍곡)이냐? <br>

A2. 식 (cosh)^2 - (sinh)^2 = 1 때문 <br>

- 원 (cos세타)^2 + (sin세타)^2 = 1 위의 한 점
> (cos세타, sin세타)

- 타원 (x/a)^2 + (y/b)^2 = 1 위의 한 점
> (a\*cos세타 b*cos세타)

- 쌍곡선 (x/a)^2 - (y/2)^2 = 1 위의 한 점
> (a\*cosh세타 b*cosh세타) <br>
> (cosh)^2 - (sinh)^2 = 1 때문

* 아다마르 프로덕트: 각 원소 별로 곱하는 것

# RNN class (모듈..?)
* ./common/time_layers.py

In [ ]:
from common.np import *  # import numpy as np (or import cupy as np)
from common.layers import *
from common.functions import sigmoid


class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next

    # 역전파 구현 코드 (진짜 그냥 수식을 그대로 구현한.)
    def backward(self, dh_next): # 역전파 하려면 흘러들어온 미분 필요. dh_next: 입력인수, 흘러들어온 미분.
                                                                # 손실함수를 h_next로 미분한 gradient일 것.
        Wx, Wh, b = self.params # 위에서 만든 리스트
                                 # Wx: 데이터 x의 weight matrix
                                 # Wh: 전 시각에서 흘러들어온 hidden_state에 대한 weight matrix
                                 # b: 편향벡터
        x, h_prev, h_next = self.cache # 순전파(forward) 할 때 만들어둔 튜플
                                 # x: input data
                                 # h_prev: 전 시각에서 흘러들어온 hidden_state
                                 # h_next: 다음 시각으로 보내는 hidden_state
        dt = dh_next * (1 - h_next ** 2) # 흘러 들어온 미분이 하이퍼볼릭탄젠트 함수를 지남.
                                          # 하이퍼볼릭탄젠트 미분 = (1-y^2);    y=h_next
                                          # 흘러들어온 미분(dh_next)과 함께 곱히기(아다마르 프로덕트)
        db = np.sum(dt, axis=0) # 하이퍼 볼릭 탄젠트를 지난 미분 dh가 덧셈노드라서 그대로 내려갈텐데,
                                # b: 편향벡터,
                                    # 만약 데이터가 배치처리를 해서 여러 장일 경우에는 편향벡터를 각 행에 그대로 카피해서 집어넣음.
                                    # 들어온 미분행렬에 대해 각 행에 대해서 카피해서 집어 넣는다 = repeat node 지나간다는 얘기
                                    # repeat node의 역전파 = sum node

                                # 들어오는 미분 행렬을 각 열에 대해 더해주기 == sum node
                                # b가 벡터이면 손실함수를 b로 미분한 gradient 역시 벡터가 돼야 함.
                                # 들어온 미분 행렬을 각 열에 대해 더해줘야 벡터가 됨
                                    # 즉, 각 열에 대해 흘러 들어온 미분을 다 더하란 소리
        dWh = np.dot(h_prev.T, dt) # 흘러들어온 미분(dt)은 덧셈노드를 지나가니까 그대로 dt
                                   # 거기에 엇갈려서 h_Transpose한 것을 곱하라는 것
        dh_prev = np.dot(dt, Wh.T) # 흘러들어온 미분 dt에 엇갈려서 Wh_Transpose해서 곱하라는 것
        dWx = np.dot(x.T, dt) # 흘러들어온 미분 dt에 엇갈려서 input data x를 x_Transpose해서 곱하라는 것
        dx = np.dot(dt, Wx.T) # 흘러들어온 미분 dt에 엇갈려서 Wx_Transpose해서 곱하라는 것

        # zero행렬들의 모음이었던 grads 역전파의 결과들로 채우기 (update)
        self.grads[0][...] = dWx # Wx에 대한 gradient
        self.grads[1][...] = dWh # Wh에 대한 gradient
        self.grads[2][...] = db # 편향벡터 b에 대한 gradient
        #### 이 층에 대한 학습은 gradient로 하게 됨.

        return dx, dh_prev # 데이터에 대한 미분, h_prev에 대한 미분
        # 밑으로 흘려 보내야 밑에 층이(전 시각에서) 학습을 할 테니까. 

In [ ]:
RNN으로 언어모델을 만들 것.
즉 단어시퀀스에다가 조건부 확률을 구현을 하겠다는...
